In [ ]:
# !pip install mediapipe opencv-python

In [42]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Angle Calculation

In [43]:
import numpy as np

In [44]:
def angle_cal(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    #vector_calculation
    v_1 = np.array([a[0]-b[0],a[1]-b[1]])
    v_2 = np.array([c[0]-b[0],c[1]-b[1]])
    
    #np.linalg.norm(v_1) = magnitude
    #a.b = a*b*cos(q)
    
    cos_theta = np.dot(v_1,v_2)/(np.linalg.norm(v_1)*np.linalg.norm(v_2))
    angle_radian = np.arccos(cos_theta)
    angle = int(np.abs(angle_radian*57.2958))
    
    if angle >180.0:
            angle = 360-angle
            
    return angle

In [45]:
angle_cal([2,3],[1,1],[4,2])

45

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [59]:
list(mp_pose.PoseLandmark)

[<PoseLandmark.NOSE: 0>,
 <PoseLandmark.LEFT_EYE_INNER: 1>,
 <PoseLandmark.LEFT_EYE: 2>,
 <PoseLandmark.LEFT_EYE_OUTER: 3>,
 <PoseLandmark.RIGHT_EYE_INNER: 4>,
 <PoseLandmark.RIGHT_EYE: 5>,
 <PoseLandmark.RIGHT_EYE_OUTER: 6>,
 <PoseLandmark.LEFT_EAR: 7>,
 <PoseLandmark.RIGHT_EAR: 8>,
 <PoseLandmark.MOUTH_LEFT: 9>,
 <PoseLandmark.MOUTH_RIGHT: 10>,
 <PoseLandmark.LEFT_SHOULDER: 11>,
 <PoseLandmark.RIGHT_SHOULDER: 12>,
 <PoseLandmark.LEFT_ELBOW: 13>,
 <PoseLandmark.RIGHT_ELBOW: 14>,
 <PoseLandmark.LEFT_WRIST: 15>,
 <PoseLandmark.RIGHT_WRIST: 16>,
 <PoseLandmark.LEFT_PINKY: 17>,
 <PoseLandmark.RIGHT_PINKY: 18>,
 <PoseLandmark.LEFT_INDEX: 19>,
 <PoseLandmark.RIGHT_INDEX: 20>,
 <PoseLandmark.LEFT_THUMB: 21>,
 <PoseLandmark.RIGHT_THUMB: 22>,
 <PoseLandmark.LEFT_HIP: 23>,
 <PoseLandmark.RIGHT_HIP: 24>,
 <PoseLandmark.LEFT_KNEE: 25>,
 <PoseLandmark.RIGHT_KNEE: 26>,
 <PoseLandmark.LEFT_ANKLE: 27>,
 <PoseLandmark.RIGHT_ANKLE: 28>,
 <PoseLandmark.LEFT_HEEL: 29>,
 <PoseLandmark.RIGHT_HEEL: 30>,
 <P

In [164]:
def left(landmarks):
    
    # Get coordinates_
    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

    shoulder_angle = angle_cal(hip, shoulder, elbow)
    elbow_angle = angle_cal(shoulder, elbow, wrist)
    
    if shoulder_angle >=70:
        shoulder_color = (20,255,57)
    else:
        shoulder_color = (0,0,255)
        
    if elbow_angle >=60:
        elbow_color = (20,255,57)
    else:
        elbow_color = (0,0,255)
        
    return [[shoulder,shoulder_angle,shoulder_color],[elbow,elbow_angle,elbow_color]]
#     return shoulder_angle, elbow_angle

def right(landmarks):
    
    # Get coordinates_
    shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]

    shoulder_angle = angle_cal(hip, shoulder, elbow)
    elbow_angle = angle_cal(shoulder, elbow, wrist)
    
    if shoulder_angle >=70:
        shoulder_color = (20,255,57)
    else:
        shoulder_color = (0,0,255)
        
    if elbow_angle >=60:
        elbow_color = (20,255,57)
    else:
        elbow_color = (0,0,255)
        
    return [[shoulder,shoulder_angle,shoulder_color],[elbow,elbow_angle,elbow_color]]
#     return shoulder_angle, elbow_angle

def visualize(arr):
    for i in arr:
        cv2.putText(image, str(i[1]), 
                           tuple(np.multiply(i[0], [int(frame_width*1.8),int(frame_height*1.6)]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, i[2], 2, cv2.LINE_AA
                                )

In [ ]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame_height, frame_width, _ = frame.shape
        frame = cv2.resize(frame,(int(frame_width*1.8),int(frame_height*1.6)))
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            visualize(left(landmarks))
            visualize(right(landmarks))
            
        except:
            pass
        
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(41,255,249), thickness=3, circle_radius=5), 
                                mp_drawing.DrawingSpec(color=(255,255,255), thickness=4, circle_radius=2) 
                                 )               
        
        cv2.imshow('Gymmify Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()